In [1]:
import os
import pathlib
import shutil
import json
import polars as pl
from pprint import pprint
from tqdm import tqdm

pl.Config.load_from_file("pl_config.json")
pl.set_random_seed(999)

## 다운로드

다운받을 데이터셋 정보 조회

In [2]:
!aihubshell -mode l -datasetkey 625

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─031.온라인 구어체 말뭉치 데이터
        ├─01.데이터
        │  └─1.Training_220728_add
        │      ├─원천데이터
        │      │  └─TS1.zip | 287 MB | 62289
        │      └─라벨링데이터
        │          └─TL1.zip | 1 GB | 62288
        ├─02.저작도구
        │  └─3.저작도구 사용매뉴얼
        │      └─저작도구 사용매뉴얼.zip | 14 MB | 69304
        └─03.AI 모델
            └─AI모델 및 소스코드.zip | 811 MB | 487495


구어체 데이터는 원천 데이터, 라벨링 데이터로 구성되어 있다. AI Hub 데이터셋 소개란에 기재된 [개요](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=625)에 따르면:

* 원천 데이터
  * 원시데이터(웹데이터)로부터 비문, 비속어, 편향성, 비식별화 등 정제된 JSON 구조의 텍스트 데이터 10억 어절 이상
* 라벨링 데이터
  * 원천데이터로부터 JSON구조의 구문에 맞게 개체명과 신조어, 형태소(간투어, 부사 등)에 대하여 태깅된 텍스트 데이터 10억 어절 이상

따라서, 언어모델 사전학습용 말뭉치 구성을 위한 텍스트만 추출해야 한다면 원천데이터(62289)만 사용하면 된다.

In [3]:
!aihubshell -mode d -datasetkey 625 -filekey 62289

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  287M    0  287M    0     0  5418k      0 --:--:--  0:00:54 --:--:-- 5659k:--:--  0:00:02 --:--:-- 4362k4 --:--:-- 4169k  0     0  3793k      0 --:--:--  0:00:07 --:--:-- 4059k 0     0  5436k      0 --:--:--  0:00:28 --:--:-- 5640k
Request successful with HTTP status 200.
Download successful.
x 031.온라인_구어체_말뭉치_데이터/01.데이터/1.Training_220728_add/원천데이터/TS1.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging TS1.zip in ./031.온라인_구어체_말뭉치_데이터/01.데이터/1.Training_220728_add/원천데이터
병합이 완료 되었습니다. 


다운받은 데이터 디렉토리 및 결과 디렉토리 정의

In [4]:
archive_dir = [path for path in pathlib.Path(os.getcwd()).glob("031*") if path.suffix != ".ipynb"][0]
result_dir = pathlib.Path(f"{os.getcwd()}/aihub-datasets")
result_dir.mkdir(parents=True, exist_ok=True)

## 압축 해제 및 EDA

원천데이터 압축파일 경로 정의

In [5]:
for walk in os.walk(archive_dir):
    current_path, folders, files = walk
    for file_name in files:
        if file_name.endswith(".zip") and current_path.endswith("원천데이터"):
            archive_path = f"{current_path}/{file_name}"

압축 해제

In [6]:
data_dir = pathlib.Path(archive_path.strip(".zip"))
shutil.unpack_archive(
    filename=archive_path,
    extract_dir=data_dir,
    format="zip",
)

각 데이터는 json 형태로 저장되어 있다. 파일 하나가 게시글 하나고, 하위 필드에 댓글 등 구어체 데이터가 달려있는 양상을 보인다.

In [7]:
data_files = list(data_dir.glob("*/*.json"))
print(f"Number of files in dataset : {len(data_files):,}")

Number of files in dataset : 23,035


데이터 속성값들

In [8]:
with open(data_files[0], "r") as file:
    documents = json.load(file)["SJML"]["text"]
pprint(documents[:4])

[{'content': '헤어스타일이, 어디선가 본듯한, 개그맨,최, 준,',
  'parent_url': 'https://www.youtube.com/watch?v=xA10nZDs0x8',
  'source_site': 'https://www.youtube.com/',
  'url': 'https://www.youtube.com/channel/UCESC_GRUe4Z7-OOxk_AWm5Q',
  'write_date': '2021-11-06',
  'writer': 'Ugxadr5R3lx6LenCsqF4AaABAg'},
 {'content': '우리 봄이도 한창 어리광 피울 나이인데 말괄량이 딸 키우느라 혼자만의 시간 필요 할거예요 헤어 스타일 치명적이네요',
  'parent_url': 'https://www.youtube.com/watch?v=xA10nZDs0x8',
  'source_site': 'https://www.youtube.com/',
  'url': 'https://www.youtube.com/channel/UCESC_GRUe4Z7-OOxk_AWm5Q',
  'write_date': '2021-11-03',
  'writer': 'UgwInLF92qcaWKROXSh4AaABAg'},
 {'content': '잘못 한거 눈치채고 도망가네 봄아. 휴지는 치워야지',
  'parent_url': 'https://www.youtube.com/watch?v=xA10nZDs0x8',
  'source_site': 'https://www.youtube.com/',
  'url': 'https://www.youtube.com/channel/UCESC_GRUe4Z7-OOxk_AWm5Q',
  'write_date': '2021-10-14',
  'writer': 'Ugz8SHOZVwxwLUBBhlh4AaABAg'},
 {'content': '저 , 궁금한게 있는데요 거실매트를 까신 (이름) 궁금해요. 혹시 강쥐들 발톱땜에 기스나서 그런건지 , 왜냐면 울집 '


전체 로드

In [9]:
corpus = []
for data_file in tqdm(data_files):
    with open(data_file, "r") as file:
        documents = json.load(file)["SJML"]["text"]
    corpus.extend([document["content"] for document in documents])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23035/23035 [00:14<00:00, 1593.39it/s]


EDA를 위해 데이터프레임으로 변환

In [10]:
eda_dataset = pl.DataFrame({"document": corpus}).sample(fraction=1, shuffle=True)
print(f"Number of documents in the corpus : {eda_dataset.height:,}")

Number of documents in the corpus : 5,133,299


### 태그가 포함된 문서 제거

위의 예시에서 `(이름)`같은 태그를 사용해 공개하기 부적절하다고 판단한 정보를 마스킹한 것을 확인할 수 있다. 위와 같은 태그가 몇 종류나 있는지 확인해보기 위해 정규표현식을 사용해 해당 패턴을 매칭하고 개수를 센다.

In [11]:
(
    eda_dataset
    .with_columns(pl.col("document").str.extract(r"(\([가-힣]+\))", 1).alias("mask_pattern"))
    .group_by("mask_pattern")
    .len("pattern_occurence_count")
    .sort(by="pattern_occurence_count", descending=True)
    .head(10)
)

mask_pattern,pattern_occurence_count
str,u32
null,"3,733,482"
"""(이름)""","775,618"
"""(비속어)""","340,288"
"""(반사회적용어)""","182,943"
"""(혐오표현)""","65,707"
"""(이메일)""","9,556"
"""(전화번호)""",365
"""(님)""",283
"""(덜렁)""",200


이메일, 전화번호까지는 필터링해야할 정보의 태그인 것으로 이해되는데, `님`, `덜렁`부터는 그런 태그라고 보기 어렵다고 봐서 예시를 확인한다.

In [12]:
eda_dataset.filter(pl.col("document").str.contains("\(님\)")).head(5)

document
str
"""엽떡이라고 하면서 노래부를때 (이름)(님)에게 못생김이 붙어있네요, 그리고 마지막 반전이네 아빠 (반사회적용어)찌개에 (반사회…"
"""저도 국민이(님)과 놀고싶내여 나도, 볼살도 만져보고 싶드아."""
"""도티(님)야.하지마라 잠뜰(님)31.31.이제 32. 한달밖에 않남았다 32.32. (이름) 다음 크리스마스는 32.32.이제…"
"""썸네일 미쳤는데 공파리파(님)이 이 썸네일을 싫어합니다."""
"""도티(님)인주세에영상을 왜이렇게 만들어..((이름) 영상너무 재미이시어여)"""


In [13]:
eda_dataset.filter(pl.col("document").str.contains("\(덜렁\)")).head(5)

document
str
"""자려고 보기시작했는데 형 썰이 너무 재밌어서 결국 끝까지 봐버렸잖아 형 책임져 사랑해 앙 (덜렁)"""
"""저는 진지한 분 좋아합니다(덜렁)"""
"""엥 랄까, 이거 완전 뜨억이 말투 아니냐 아무튼 (반사회적용어)텽보고 불평하지마라. 쒸이, (덜렁)"""
"""형 사랑해(덜렁)"""
"""그 더러운 손 치우시지. 가자 왁굳형 (덜렁)"""


문서 정제 과정에서 생긴 태그가 아니라고 보고, 위에서 `(이름)`부터 `(전화번호)`까지 총 6개 태그가 포함된 문서들을 제거한다.

In [14]:
eda_dataset = eda_dataset.filter(~pl.col("document").str.contains(r"(\(이름|비속어|반사회적용어|혐오표현|이메일|전화번호\))"))
print(f"Number of documents in the corpus after filtering : {eda_dataset.height:,}")

Number of documents in the corpus after filtering : 3,754,893


### 문서 토큰 수 분포 확인

한국어에서는 정확히 적용되지 않지만, 편의를 위해 어절의 수(`공백 수 + 1`)를 토큰 수라고 정의한다. 개요에는 원시데이터에 10억개 이상의 어절이 포함되어 있다고 적혀있었지만, 이 노트북의 기준에 따라 사용할만한 문서에 포함된 어절의 수는 2천만개 가량에 그친다는 것을 알 수 있다.

In [15]:
eda_dataset = eda_dataset.with_columns(pl.col("document").str.replace(r"\s+", " "))
num_tokens = (
    eda_dataset
    .with_columns((pl.col("document").str.count_matches(" ") + 1).alias("num_tokens"))
    .select("num_tokens")
)
print(f"Number of tokens in the corpus : {num_tokens['num_tokens'].sum():,}")

Number of tokens in the corpus : 22,826,166


문서당 토큰 수 분포. 절반 이상이 4개 내외의 토큰으로 이루어진 짧은 문서임을 알 수 있다.

In [16]:
num_tokens.select(
    pl.col("num_tokens").min().alias("min"),
    pl.col("num_tokens").quantile(0.05).cast(pl.Int32).alias("q05"),
    pl.col("num_tokens").quantile(0.25).cast(pl.Int32).alias("Q1"),
    pl.col("num_tokens").quantile(0.50).cast(pl.Int32).alias("median"),
    pl.col("num_tokens").quantile(0.75).cast(pl.Int32).alias("Q3"),
    pl.col("num_tokens").quantile(0.95).cast(pl.Int32).alias("q95"),
    pl.col("num_tokens").max().alias("max"),
)

min,q05,Q1,median,Q3,q95,max
u32,i32,i32,i32,i32,i32,u32
2,2,3,4,7,16,"1,440"


## 데이터 저장

EDA를 마친 corpus를 앞서 정의한 `result_dir`에 같은 다운받은 데이터셋과 동일한 이름으로 저장

In [17]:
result_path = result_dir.joinpath(f"{archive_dir.name}.parquet")
eda_dataset.write_parquet(result_path)

로컬 디렉토리 정리

In [18]:
os.system(f"rm -rf {archive_dir}")

0